<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# TROLL HUNTING: DETECTING STATE-BACKED DISINFORMATION CAMPAIGNS ON TWITTER

# NOTEOOK 1.5:
I decided to add this notebook after reviewing the initial rounds of results of my Logistic Regression model. More will be explained later, but here I'll create another training set of fake tweets, this time from Iran, to build a model that is trained on Iranian troll tweets (plus real tweets from American verified users). 

It would be too unwieldly to incorporate this in the earlier notebooks.  

The dataset generated here will be used in notebook 3.2

In [1]:
from langdetect import detect
from textblob import TextBlob

import pandas as pd
import re

In [2]:
# NOTE: This CSV is NOT included in this repo due to the file size. 
# Download via earlier link shared from Twitter's Elections Integrity site
iran = pd.read_csv('../data/iranian_tweets.csv')

In [3]:
iran.shape

(1122936, 31)

In [4]:
# I'm following a similar set of cleaning routines in the earlier notebook for Russian state-backed tweets
# I'm focusing only on English tweets as well as non-RTs
iran = iran.drop(
    columns=[
        "userid",
        "user_display_name",
        "user_profile_url",
        "tweet_client_name",
        "in_reply_to_tweetid",
        "in_reply_to_userid",
        "quoted_tweet_tweetid",
        "is_retweet",
        "retweet_userid",
        "retweet_tweetid",
        "latitude",
        "longitude",
        "quote_count",
        "reply_count",
        "like_count",
        "retweet_count",
        "urls",
        "user_mentions",
        "poll_choices",
        "hashtags",
        "account_language",
        "tweet_language"
    ]
)

In [5]:
iran_tweets = iran.sample(n=1000000, random_state=42)

In [6]:
def detect_language_langdetect(text):
    try:
        return detect(text)
    except:
        return 'unk'

In [7]:
def detect_language_textblob(text):
    try:
        return TextBlob(text).detect_language
    except:
        return 'unk'

In [8]:
iran_tweets['lang_textblob'] = iran_tweets['tweet_text'].apply(detect_language_textblob)
iran_tweets['lang_textblob_loc'] = iran_tweets['user_reported_location'].apply(detect_language_textblob)

In [9]:
iran_tweets['langdetect'] = iran_tweets['tweet_text'].apply(detect_language_langdetect)
iran_tweets['langdetect_loc'] = iran_tweets['user_reported_location'].apply(detect_language_langdetect)

In [10]:
iran_tweets = iran_tweets[(iran_tweets['langdetect'] == 'en') & (iran_tweets['langdetect_loc'] == 'en')].copy()

In [11]:
iran_tweets['langdetect'].value_counts()

en    64767
Name: langdetect, dtype: int64

In [12]:
iran_tweets = iran_tweets[~iran_tweets['tweet_text'].str.startswith("RT @")].copy()

In [13]:
iran_tweets = iran_tweets.drop(columns=['langdetect', 'langdetect_loc', 'lang_textblob', 'lang_textblob_loc'])

In [14]:
iran_tweets = iran_tweets[
    [
        "tweetid",
        "user_screen_name",
        "user_reported_location",
        "user_profile_description",
        "follower_count",
        "following_count",
        "account_creation_date",
        "tweet_time",
        "tweet_text",
    ]
]

In [15]:
iran_tweets = iran_tweets.dropna()

In [16]:
iran_tweets.shape

(37294, 9)

In [32]:
# NOTE: This CSV is included in the repo
#iran_full_en = iran_tweets.to_csv('../data/iran_full_en.csv')

#### This is the end of the section on data collection and formatting.